<a href="https://colab.research.google.com/github/poring3995/My_Project/blob/MY_Project/Project_classification_(BigDL_DLlib).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bigdl-dllib-spark3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 MB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 22.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.3-py2.py3-none-any.whl size=214463469 sha256=bb20e6a308de0497aa5494cfb9568f349c0c7941f897913d26d1819b4460dbfb
  Stored in directory: /root/.cache/pip/wheels/b0/d0/fd/b0e0165f0fbd79838d2f50c544382e7a5346274f0af07ffada
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from bigdl.dllib.nncontext import *
from bigdl.dllib.keras.layers import *
from bigdl.dllib.keras.models import *
import bigdl.dllib.keras.Sequential
from bigdl.dllib.nnframes import *
from bigdl.dllib.nn.criterion import *

In [ ]:
sc = init_nncontext(cluster_mode="local")
spark = SparkSession(sc)

Current pyspark location is : /usr/local/lib/python3.10/dist-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /usr/local/lib/python3.10/dist-packages/bigdl/share/dllib/lib/bigdl-dllib-spark_3.1.3-2.3.0-jar-with-dependencies.jar:/usr/local/lib/python3.10/dist-packages/bigdl/share/core/lib/all-2.3.0.jar pyspark-shell 
Successfully got a SparkContext


In [ ]:
df = spark.read.csv('/content/star_classification.csv', header='true', inferSchema='true')
df.show()

+--------------------+----------------+------------------+--------+--------+--------+--------+--------+------+--------+-------+--------+--------------------+------+------------+-----+-----+--------+
|              obj_ID|           alpha|             delta|       u|       g|       r|       i|       z|run_ID|rerun_ID|cam_col|field_ID|         spec_obj_ID| class|    redshift|plate|  MJD|fiber_ID|
+--------------------+----------------+------------------+--------+--------+--------+--------+--------+------+--------+-------+--------+--------------------+------+------------+-----+-----+--------+
|1.237660961327743...|  135.6891066036|  32.4946318397087|23.87882| 22.2753|20.39501|19.16573|18.79371|  3606|     301|      2|      79|6.543777369295181...|GALAXY|   0.6347936| 5812|56354|     171|
|1.237664879951151...|144.826100550256|  31.2741848944939|24.77759|22.83188|22.58444|21.16812|21.61427|  4518|     301|      5|     119|1.176014203670733...|GALAXY|    0.779136|10445|58158|     427|
|1.23

# **Datapreprocessing :** Since the data in the class column is in the type string,so we can't applies these data into training (in the case that we didn't change it the predictio value from the training set will be only NULL)

# So, we convert the the data in string type into the numeric by setting the condition for each categories into the number (don't worry about the string type first becuase it will be get changetype in the training data setting)

# **GALAXY into "1"**
# **SQO into "2"**
# **STAR into "3"**

In [ ]:
from pyspark.sql.functions import when
df_transform = df.withColumn("class", when(df["class"] == "GALAXY","1") \
      .when(df["class"] == "QSO","2") \
      .otherwise("3"))

df_transform.show()


+--------------------+----------------+------------------+--------+--------+--------+--------+--------+------+--------+-------+--------+--------------------+-----+------------+-----+-----+--------+
|              obj_ID|           alpha|             delta|       u|       g|       r|       i|       z|run_ID|rerun_ID|cam_col|field_ID|         spec_obj_ID|class|    redshift|plate|  MJD|fiber_ID|
+--------------------+----------------+------------------+--------+--------+--------+--------+--------+------+--------+-------+--------+--------------------+-----+------------+-----+-----+--------+
|1.237660961327743...|  135.6891066036|  32.4946318397087|23.87882| 22.2753|20.39501|19.16573|18.79371|  3606|     301|      2|      79|6.543777369295181...|    1|   0.6347936| 5812|56354|     171|
|1.237664879951151...|144.826100550256|  31.2741848944939|24.77759|22.83188|22.58444|21.16812|21.61427|  4518|     301|      5|     119|1.176014203670733...|    1|    0.779136|10445|58158|     427|
|1.2376609

In [ ]:
df = df_transform

# Setting feature of by using vecAssembler (the input column is same as we use in the precious algorithm)

In [ ]:
vecAssembler = VectorAssembler(outputCol="features")
vecAssembler.setInputCols(["alpha","delta","u","g","r","i","z","redshift"])
train_df = vecAssembler.transform(df)
changedTypedf = train_df.withColumn("label", train_df["class"].cast(DoubleType()))\
    .select("features", "label")(trainingDF, validationDF) = changedTypedf.randomSplit([0.7, 0.3])

In [ ]:
trainingDF.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.05101121242955...|  2.0|
|[0.05249126170593...|  1.0|
|[0.07104565882764...|  1.0|
|[0.07573854975100...|  3.0|
|[0.07960455239971...|  1.0|
|[0.08104328851311...|  1.0|
|[0.08494855971241...|  2.0|
|[0.08524942209373...|  2.0|
|[0.09206793706016...|  2.0|
|[0.10189312998176...|  2.0|
|[0.10724335454108...|  3.0|
|[0.10899016498331...|  2.0|
|[0.11599688273469...|  2.0|
|[0.12635265002273...|  1.0|
|[0.14102713378031...|  3.0|
|[0.14673517170572...|  2.0|
|[0.15236163796890...|  3.0|
|[0.16142408327709...|  3.0|
|[0.18224373885289...|  1.0|
|[0.20530953273652...|  3.0|
+--------------------+-----+
only showing top 20 rows



# Design the model

In [ ]:
x1 = Input(shape=(8,))
dense1 = Dense(12, activation='relu')(x1)
dense2 = Dense(6, activation='relu')(dense1)
dense3 = Dense(3)(dense2)
model = Model(x1, dense3)

creating: createZooKerasInput
creating: createZooKerasDense
creating: createZooKerasDense
creating: createZooKerasDense
creating: createZooKerasModel


# for tuning parameter, I do a some experiment to find the best parameter that can give the good score of accuracy also not takes too long to get the result

# here is my record :

batch size 100 epoch 20 acc 0.949

batch size 100 epoch 50 acc 0.9401

batch size 100 epoch 100 acc 09556 (it takes too long to get result)

batch size 200 epoch 50 acc 0.9481

batch size 200 epoch 100 acc 0.9496

batch size 200 epoch 150 acc 0.955584

# as from my record the optimal parameter would be batch size 200 with 150 epoch for trainning since it still give high accuracy with slighty less processing time

In [ ]:
classifier = NNClassifier(model, CrossEntropyCriterion(), [8]) \
    .setOptimMethod(Adam()) \
    .setBatchSize(200) \
    .setMaxEpoch(150)

creating: createCrossEntropyCriterion
creating: createScalarToTensor
creating: createSeqToTensor
creating: createFeatureLabelPreprocessing
creating: createNNClassifier
creating: createAdam


In [ ]:
nnModel = classifier.fit(trainingDF)

creating: createToTuple
creating: createChainedPreprocessing


# Prediction result

In [ ]:
predictionDF = nnModel.transform(validationDF).cache()
predictionDF.sample(False, 0.1).show()

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictionDF)

print("Accuracy = %g " % (accuracy))

+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|[0.05358176835898...|  2.0|       2.0|
|[0.09356911689565...|  1.0|       1.0|
|[0.54324894205989...|  3.0|       3.0|
|[0.57956834695488...|  1.0|       1.0|
|[0.67842711058780...|  1.0|       1.0|
|[0.79527841251427...|  1.0|       1.0|
|[1.21321162204734...|  1.0|       1.0|
|[1.23696751486091...|  1.0|       1.0|
|[1.41793979925058...|  1.0|       1.0|
|[1.54551627407905...|  1.0|       1.0|
|[1.58893035584146...|  1.0|       1.0|
|[1.93660317371092...|  1.0|       1.0|
|[2.14644524048299...|  1.0|       1.0|
|[2.21779531276553...|  3.0|       3.0|
|[2.72570670343663...|  2.0|       2.0|
|[2.94045476914954...|  2.0|       2.0|
|[3.23072191407536...|  1.0|       1.0|
|[3.44613071657875...|  1.0|       1.0|
|[4.47006550932283...|  1.0|       1.0|
|[4.58886003315289...|  1.0|       2.0|
+--------------------+-----+----------+
only showing top 20 rows

Accuracy = 0.9